# Limpieza cuestionario Escuelas

In [1]:
##LECTURA DEL FICHERO SPSS Y SUS METADATOS
##Debe estar instalado el paquete de pyreadstat que permite leer ficheros SPSS y SAS
import pyreadstat
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')

pd.options.display.max_columns = 50

In [2]:
#
data, meta = pyreadstat.pyreadstat.read_sav('escuelasESP1.sav')

### Bucle que elimina todas las columnas con todos sus valores a nulos
### Parece que la mayoria son las columnas que empiezan por PA sobre padres, puede que estos valores
### se encuentren en el cuestionario para padres que es otro fichero

In [3]:
#Se recorren las columnas del df, y sí todos los valores son nulos se añaden a una lista de columnas
#que posteriormente se borran del dataframe
lis_col = []
for col in data.columns:
    val_nul = data[col].isnull().sum()
    if val_nul == data.shape[0]:
       print('columna con todo valores nulos ', col)
       lis_col.append(col)
data2 = data.drop(lis_col, axis=1)

columna con todo valores nulos  BOOKID


In [4]:
data2.columns

Index(['CNTRYID', 'CNT', 'CNTSCHID', 'CYC', 'NatCen', 'Region', 'STRATUM',
       'SUBNATIO', 'OECD', 'ADMINMODE',
       ...
       'CREACTIV', 'EDUSHORT', 'STAFFSHORT', 'STUBEHA', 'TEACHBEHA', 'SCMCEG',
       'W_SCHGRNRABWT', 'W_FSTUWT_SCH_SUM', 'SENWT', 'VER_DAT'],
      dtype='object', length=196)

### Seleccionamos las columnas en las que todos sus valores sean iguales para eliminarlas
### Por lo que se ve son codigos de pais en su mayoria

In [5]:
#Buscamos las columnas en las que todos los valores sean los mismos
des_data2 = data2.astype('object').describe(include = 'all').transpose()
des_data2[des_data2["unique"] == 1.0].transpose()

,CNTRYID,CNT,CYC,NatCen,SUBNATIO,OECD,ADMINMODE,SC053D11TA,VER_DAT
count,1089.0,1089,1089,1089,1089,1089.0,1089.0,1089,1089
unique,1.0,1,1,1,1,1.0,1.0,1,1
top,724.0,ESP,07MS,072400,7240000,1.0,2.0,9999997,26APR19:08:54:10
freq,1089.0,1089,1089,1089,1089,1089.0,1089.0,1089,1089


In [6]:
#Eliminamos dichas columnas ya que no daran información
lis_col2 = des_data2[des_data2["unique"] == 1.0].transpose().columns
data3 = data2.drop(lis_col2, axis=1)

# Comiento análisis y transformación resto variables


In [7]:
## generamos el dataframe data4 con los id de las escueles
data4 = pd.DataFrame(data3.loc[:, ['CNTSCHID']])

In [8]:
data['SC001Q01TA']

0       5.0
1       NaN
2       3.0
3       4.0
4       3.0
       ... 
1084    3.0
1085    2.0
1086    3.0
1087    3.0
1088    1.0
Name: SC001Q01TA, Length: 1089, dtype: float64

In [9]:
##Cargar los códigos de la comunidad autónoma y población
data4['CCAA'] = data3['Region'].astype(str).str[3:5]
data4['SC001Q01TA'] = data3['SC001Q01TA']
data4['SC013Q01TA'] = data3['SC013Q01TA']

# DESDE ESTE PUNTO VIENDO QUE LAS MAYORIA DE LAS TRANSFORMACIONES SON IGUALES, REALIZO LAS TRANSFORMACIONES EN GRUPO, MEDIANTE LA SIGUIENTE FUNCION

In [10]:
#Recorremos todas las variables e imputamos el valor máximo a los NaN
def Transfor (var, data3, data4):
    for var in vari:
        a = data3[var].value_counts(dropna=False)
        e = a.idxmax()
        #Imputamos los valores nulos al máximo
        data3.fillna({var:e}, inplace=True)
        ##Dicotomizamos y transformamos el nombre de las columnas con el prefijo del nombre de la columna y su 'valor'
        dummies = pd.get_dummies(data3[var])
        for nom_col in dummies:
            dummies = dummies.rename(columns={nom_col:var + '_' +str(nom_col)})
        # Al tener valores si o no cruzamos por indice y unimos los dataframe 
        data4 = pd.merge(data4, dummies, left_index=True, right_index=True)
    return data4

In [11]:
#data4.groupby(['CCAA']).mean()

## SC017Q01NA	School's instruction hindered by: A lack of teaching staff.
## SC017Q02NA	School's instruction hindered by: Inadequate or poorly qualified teaching staff.
## SC017Q03NA	School's instruction hindered by: A lack of assisting staff.
## SC017Q04NA	School's instruction hindered by: Inadequate or poorly qualified assisting staff.
## SC017Q05NA	School's instruction hindered by: A lack of educational material [...]
## SC017Q06NA	School's instruction hindered by: Inadequate or poor quality educational material [...]
## SC017Q07NA	School's instruction hindered by: A lack of physical infrastructure [...]
## SC017Q08NA	School's instruction hindered by: Inadequate or poor quality physical infrastructure [...]
## SC161Q01SA	Main responsibility for career guidance at school: Not applicable, career guidance is not available in this school
## SC161Q02SA	Main responsibility for career guidance at school: All teachers share the responsibility for career guidance
## SC161Q03SA	Main responsibility for career guidance at school: Specific teachers have the main responsibility for career guidance
## SC161Q04SA	Main responsibility for career guidance at school: We have [...] career guidance counsellors employed at school
## SC161Q05SA	Main responsibility for career guidance at school: We have [...] career guidance counsellors who regularly visit [...]
## SC162Q01SA	If career guidance is available at your school, which of the statements below best describes the situation for [...]
## SC155Q01HA	School's capacity using digital devices: The number of digital devices connected to the Internet is sufficient
## SC155Q02HA	School's capacity using digital devices: The school's Internet bandwidth or speed is sufficient
## SC155Q03HA	School's capacity using digital devices: The number of digital devices for instruction is sufficient
## SC155Q04HA	School's capacity using digital devices: Digital devices [...] are sufficiently powerful in terms of computing capacity
## SC155Q05HA	School's capacity using digital devices: The availability of adequate software is sufficient
## SC155Q06HA	School's capacity using digital devices: Teachers have the [...] skills to integrate digital devices in instruction
## SC155Q07HA	School's capacity using digital devices: Teachers have sufficient time to prepare lessons integrating digital devices
## SC155Q08HA	School's capacity using digital devices: Effective professional resources for teachers to learn how to use digital [...]
## SC155Q09HA	School's capacity using digital devices: An effective online learning support platform is available
## SC155Q10HA	School's capacity using digital devices: Teachers are provided with incentives to integrate digital devices in [...]
## SC155Q11HA	School's capacity using digital devices: The school has sufficient qualified technical assistant staff
## SC001Q01TA	Which of the following definitions best describes the community in which your school is located?
## SC013Q01TA	Is your school a public or a private school?



In [12]:
vari = ['SC017Q01NA',
'SC017Q02NA',
'SC017Q03NA',
'SC017Q04NA',
'SC017Q05NA',
'SC017Q06NA',
'SC017Q07NA',
'SC017Q08NA',
'SC161Q01SA',
'SC161Q02SA',
'SC161Q03SA',
'SC161Q04SA',
'SC161Q05SA',
'SC162Q01SA',
'SC155Q01HA',
'SC155Q02HA',
'SC155Q03HA',
'SC155Q04HA',
'SC155Q05HA',
'SC155Q06HA',
'SC155Q07HA',
'SC155Q08HA',
'SC155Q09HA',
'SC155Q10HA',
'SC001Q01TA',
'SC013Q01TA']


In [13]:
for var in vari:
    print(meta.variable_value_labels[var])

{1.0: 'Not at all', 2.0: 'Very little', 3.0: 'To some extent', 4.0: 'A lot', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Not at all', 2.0: 'Very little', 3.0: 'To some extent', 4.0: 'A lot', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Not at all', 2.0: 'Very little', 3.0: 'To some extent', 4.0: 'A lot', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Not at all', 2.0: 'Very little', 3.0: 'To some extent', 4.0: 'A lot', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Not at all', 2.0: 'Very little', 3.0: 'To some extent', 4.0: 'A lot', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Not at all', 2.0: 'Very little', 3.0: 'To some extent', 4.0: 'A lot', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Not at all', 2.0: 'Very little', 3.0: 'To some extent', 4.0: 'A

In [14]:
for var in vari:
    print(data3[var].value_counts(dropna=False))

1.0    386
3.0    337
2.0    262
4.0     60
NaN     44
Name: SC017Q01NA, dtype: int64
1.0    449
2.0    369
3.0    207
NaN     47
4.0     17
Name: SC017Q02NA, dtype: int64
3.0    422
2.0    262
4.0    183
1.0    179
NaN     43
Name: SC017Q03NA, dtype: int64
1.0    562
2.0    324
3.0    131
NaN     49
4.0     23
Name: SC017Q04NA, dtype: int64
1.0    381
2.0    304
3.0    283
4.0     82
NaN     39
Name: SC017Q05NA, dtype: int64
1.0    422
2.0    343
3.0    216
4.0     64
NaN     44
Name: SC017Q06NA, dtype: int64
1.0    338
3.0    295
2.0    290
4.0    128
NaN     38
Name: SC017Q07NA, dtype: int64
1.0    333
2.0    314
3.0    273
4.0    128
NaN     41
Name: SC017Q08NA, dtype: int64
0.0    1054
NaN      32
1.0       3
Name: SC161Q01SA, dtype: int64
0.0    841
1.0    216
NaN     32
Name: SC161Q02SA, dtype: int64
0.0    697
1.0    360
NaN     32
Name: SC161Q03SA, dtype: int64
1.0    908
0.0    149
NaN     32
Name: SC161Q04SA, dtype: int64
0.0    995
1.0     62
NaN     32
Name: SC161Q05SA, dt

In [15]:
data4 = Transfor(vari, data3, data4)

## SC156Q01HA	At school: Its own written statement about the use of digital devices
## SC156Q02HA	At school: Its own written statement specifically about the use of digital devices for pedagogical purposes
## SC156Q03HA	At school: A programme to use digital devices for teaching and learning in specific subjects
## SC156Q04HA	At school: Regular discussions with teaching staff about the use of digital devices for pedagogical purposes
## SC156Q05HA	At school: A specific programme to prepare students for responsible Internet behaviour
## SC156Q06HA	At school: A specific policy about using Social Networks (Facebook, etc.) in teaching and learning
## SC156Q07HA	At school: A specific programme to promote collaboration on the use of digital devices among teachers
## SC156Q08HA	At school: Scheduled time for teachers to meet to share, evaluate or develop instructional materials and [...]
## SC011Q01TA	Which of the following statements best describes the schooling available to students in your location?
## SC012Q01TA	Student admission to school: Student's record of academic performance (including placement tests)
## SC012Q02TA	Student admission to school: Recommendation of feeder schools
## SC012Q03TA	Student admission to school: Parents' endorsement of the instructional or religious philosophy of the school
## SC012Q04TA	Student admission to school: Whether the student requires or is interested in a special programme
## SC012Q05TA	Student admission to school: Preference given to family members of current or former students
## SC012Q06TA	Student admission to school: Residence in a particular area
## SC012Q07TA	Student admission to school: Other
## SC042Q01TA	School's policy for national modal grade for 15-year-olds: Students are grouped by ability into different classes.
## SC042Q02TA	School's policy for national modal grade for 15-year-olds: Students are grouped by ability within their classes.
## SC154Q01HA	School's use of assessments of students: To guide students' learning
## SC154Q02WA	School's use of assessments of students: To inform parents about their child's progress
## SC154Q03WA	School's use of assessments of students: To make decisions about students' retention or promotion
## SC154Q04WA	School's use of assessments of students: To group students for instructional purposes
## SC154Q05WA	School's use of assessments of students: To compare the school to district or national performance
## SC154Q06WA	School's use of assessments of students: To monitor the school’s progress from year to year
## SC154Q07WA	School's use of assessments of students: To make judgements about teachers' effectiveness
## SC154Q08WA	School's use of assessments of students: To identify aspects of instruction or the curriculum that could be improved
## SC154Q09HA	School's use of assessments of students: To adapt teaching to the students' needs
## SC154Q10WA	School's use of assessments of students: To compare the school with other schools
## SC154Q11HA	School's use of assessments of students: To award certificates to students


In [16]:
vari = ['SC156Q01HA',
'SC156Q02HA',
'SC156Q03HA',
'SC156Q04HA',
'SC156Q05HA',
'SC156Q06HA',
'SC156Q07HA',
'SC156Q08HA',
'SC011Q01TA',
'SC012Q01TA',
'SC012Q02TA',
'SC012Q03TA',
'SC012Q04TA',
'SC012Q05TA',
'SC012Q06TA',
'SC012Q07TA',
'SC042Q01TA',
'SC042Q02TA',
'SC154Q01HA',
'SC154Q02WA',
'SC154Q03WA',
'SC154Q04WA',
'SC154Q05WA',
'SC154Q06WA',
'SC154Q07WA',
'SC154Q08WA',
'SC154Q09HA',
'SC154Q10WA',
'SC154Q11HA']


In [17]:
for var in vari:
    print(meta.variable_value_labels[var])

{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'There are two or more other schools in this area that compete for our students.', 2.0: 'There is one other school in this area that competes for our students.', 3.0: 'There are 

In [18]:
for var in vari:
    print(data3[var].value_counts(dropna=False))

1.0    611
2.0    440
NaN     38
Name: SC156Q01HA, dtype: int64
2.0    690
1.0    362
NaN     37
Name: SC156Q02HA, dtype: int64
2.0    661
1.0    386
NaN     42
Name: SC156Q03HA, dtype: int64
2.0    688
1.0    360
NaN     41
Name: SC156Q04HA, dtype: int64
1.0    561
2.0    489
NaN     39
Name: SC156Q05HA, dtype: int64
2.0    533
1.0    518
NaN     38
Name: SC156Q06HA, dtype: int64
2.0    724
1.0    323
NaN     42
Name: SC156Q07HA, dtype: int64
2.0    818
1.0    233
NaN     38
Name: SC156Q08HA, dtype: int64
1.0    727
2.0    173
3.0    151
NaN     38
Name: SC011Q01TA, dtype: int64
1.0    879
2.0    124
NaN     53
3.0     33
Name: SC012Q01TA, dtype: int64
1.0    859
2.0    121
NaN     55
3.0     54
Name: SC012Q02TA, dtype: int64
1.0    797
3.0    121
2.0    119
NaN     52
Name: SC012Q03TA, dtype: int64
1.0    607
2.0    277
3.0    153
NaN     52
Name: SC012Q04TA, dtype: int64
3.0    419
1.0    366
2.0    253
NaN     51
Name: SC012Q05TA, dtype: int64
3.0    592
1.0    274
2.0    183
NaN  

In [19]:
data4 = Transfor(vari, data3, data4)

## SC036Q01TA	Use of achievement data in school: Achievement data are posted publicly (e.g. in the media)
## SC036Q02TA	Use of achievement data in school: Achievement data are tracked over time by an administrative authority
## SC036Q03NA	Use of achievement data in school: Achievement data are provided directly to parents
## SC037Q01TA	Quality assurance at school: Internal evaluation/Self-evaluation
## SC037Q02TA	Quality assurance at school: External evaluation
## SC037Q03TA	Quality assurance at school: Written specification of the school's curricular profile and educational goals
## SC037Q04TA	Quality assurance at school: Written specification of student performance standards
## SC037Q05NA	Quality assurance at school: Systematic recording of data such as [...] attendance and professional development
## SC037Q06NA	Quality assurance at school: Systematic recording of student test results and graduation rates
## SC037Q07TA	Quality assurance at school: Seeking written feedback from students (e.g. regarding lessons, teachers or resources)
## SC037Q08TA	Quality assurance at school: Teacher mentoring
## SC037Q09TA	Quality assurance at school: Regular consultation aimed at school improvement [...] over a period of at least six months
## SC037Q10NA	Quality assurance at school: Implementation of a standardised policy for reading subjects [...]
## SC165Q01HA	Teachers' practices: [...] students learn about the histories of diverse cultural groups that live in country of test.
## SC165Q02HA	Teachers' practices: [...] students learn about the histories of diverse cultural groups that live in other countries.
## SC165Q03HA	Teachers' practices: [...] students learn about the cultures [...] of [...] groups that live in country of test
## SC165Q04HA	Teachers' practices: [...] students learn about different cultural perspectives on historical and social events.
## SC165Q05HA	Teachers' practices: Our school supports activities that encourage students’ expression of diverse identities [...]
## SC165Q06HA	Teachers' practices: Our school offers an exchange programme with schools in other countries.
## SC165Q07HA	Teachers' practices: Our school organises multicultural events (e.g. cultural diversity day).
## SC165Q08HA	Teachers' practices: In our school, we celebrate festivities from other cultures.
## SC165Q09HA	Teachers' practices: In our school, students are encouraged to communicate with people from other cultures via [...]
## SC165Q10HA	Teachers' practices: Our school adopts different approaches to educate students about cultural differences [...]
## SC166Q02HA	Opinion shared by teaching staff: It is important for students to learn that people from other cultures can have [...]
## SC166Q03HA	Opinion shared by teaching staff: Respecting other cultures is something that students should learn as early as possible
## SC166Q05HA	Opinion shared by teaching staff: In the classroom, it is important that students of different origins recognise [...]
## SC166Q06HA	Opinion shared by teaching staff: When there are conflicts between students of different origins, they should be [...]


In [20]:
vari = ['SC036Q01TA',
'SC036Q02TA',
'SC036Q03NA',
'SC037Q01TA',
'SC037Q02TA',
'SC037Q03TA',
'SC037Q04TA',
'SC037Q05NA',
'SC037Q06NA',
'SC037Q07TA',
'SC037Q08TA',
'SC037Q09TA',
'SC037Q10NA',
'SC165Q01HA',
'SC165Q02HA',
'SC165Q03HA',
'SC165Q04HA',
'SC165Q05HA',
'SC165Q06HA',
'SC165Q07HA',
'SC165Q08HA',
'SC165Q09HA',
'SC165Q10HA',
'SC166Q02HA',
'SC166Q03HA',
'SC166Q05HA',
'SC166Q06HA']


In [21]:
for var in vari:
    print(meta.variable_value_labels[var])

{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes, this is mandatory, e.g. based on district or ministry policies', 2.0: 'Yes, based on school initiative', 3.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes, this is mandatory, e.g. based on district or ministry policies', 2.0: 'Yes, based on school initiative', 3.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes, this is mandatory, e.g. based on district or ministry policies', 2.0: 'Yes, based on school initiative', 3.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes, this is mandatory, e.g. based on district or ministry policies',

In [22]:
for var in vari:
    print(data3[var].value_counts(dropna=False))

2.0    969
1.0     82
NaN     38
Name: SC036Q01TA, dtype: int64
1.0    791
2.0    257
NaN     41
Name: SC036Q02TA, dtype: int64
1.0    870
2.0    182
NaN     37
Name: SC036Q03NA, dtype: int64
2.0    536
1.0    386
3.0    130
NaN     37
Name: SC037Q01TA, dtype: int64
1.0    693
3.0    236
2.0    125
NaN     35
Name: SC037Q02TA, dtype: int64
1.0    630
2.0    292
3.0    106
NaN     61
Name: SC037Q03TA, dtype: int64
1.0    431
2.0    353
3.0    258
NaN     47
Name: SC037Q04TA, dtype: int64
1.0    578
2.0    380
3.0     92
NaN     39
Name: SC037Q05NA, dtype: int64
1.0    568
2.0    418
3.0     66
NaN     37
Name: SC037Q06NA, dtype: int64
2.0    691
3.0    282
1.0     75
NaN     41
Name: SC037Q07TA, dtype: int64
3.0    699
2.0    270
1.0     69
NaN     51
Name: SC037Q08TA, dtype: int64
3.0    738
2.0    261
1.0     49
NaN     41
Name: SC037Q09TA, dtype: int64
3.0    589
2.0    354
1.0    106
NaN     40
Name: SC037Q10NA, dtype: int64
1.0    807
2.0    244
NaN     38
Name: SC165Q01HA, dtype: 

In [23]:
data4 = Transfor(vari, data3, data4)

## SC167Q01HA	Curriculum for the following in <national modal grade for 15-year-olds>: Communicating with people from different [...]
## SC167Q02HA	Curriculum for the following in national modal grade for 15-year-olds: Knowledge of different cultures
## SC167Q03HA	Curriculum for the following in national modal grade for 15-year-olds: Openness to intercultural experiences
## SC167Q04HA	Curriculum for the following in national modal grade for 15-year-olds: Respect for cultural diversity
## SC167Q05HA	Curriculum for the following in national modal grade for 15-year-olds: Foreign languages
## SC167Q06HA	Curriculum for the following in national modal grade for 15-year-olds: Critical thinking skills
## SC158Q01HA	Formal curriculum in national modal grade for 15-year-olds: Climate change and global warming
## SC158Q02HA	Formal curriculum in national modal grade for 15-year-olds: Global health (e.g. epidemics)
## SC158Q04HA	Formal curriculum in national modal grade for 15-year-olds: Migration (movement of people)
## SC158Q07HA	Formal curriculum in national modal grade for 15-year-olds: International conflicts
## SC158Q08HA	Formal curriculum in national modal grade for 15-year-olds: Hunger or malnutrition in different parts of the world
## SC158Q09HA	Formal curriculum in national modal grade for 15-year-olds: Causes of poverty
## SC158Q12HA	Formal curriculum in national modal grade for 15-year-olds: Equality between men and women in [...] the world
## SC061Q01TA	Extent to which student learning is hindered by: Student truancy
## SC061Q02TA	Extent to which student learning is hindered by: Students skipping classes
## SC061Q03TA	Extent to which student learning is hindered by: Students lacking respect for teachers
## SC061Q04TA	Extent to which student learning is hindered by: Student use of alcohol or illegal drugs
## SC061Q05TA	Extent to which student learning is hindered by: Students intimidating or bullying other students
## SC061Q11HA	Extent to which student learning is hindered by: Students not being attentive
## SC061Q06TA	Extent to which student learning is hindered by: Teachers not meeting individual students' needs
## SC061Q07TA	Extent to which student learning is hindered by: Teacher absenteeism
## SC061Q08TA	Extent to which student learning is hindered by: Staff resisting change
## SC061Q09TA	Extent to which student learning is hindered by: Teachers being too strict with students
## SC061Q10TA	Extent to which student learning is hindered by: Teachers not being well prepared for classes


In [24]:
vari = ['SC167Q01HA',
'SC167Q02HA',
'SC167Q03HA',
'SC167Q04HA',
'SC167Q05HA',
'SC167Q06HA',
'SC158Q01HA',
'SC158Q02HA',
'SC158Q04HA',
'SC158Q07HA',
'SC158Q08HA',
'SC158Q09HA',
'SC158Q12HA',
'SC061Q01TA',
'SC061Q02TA',
'SC061Q03TA',
'SC061Q04TA',
'SC061Q05TA',
'SC061Q11HA',
'SC061Q06TA',
'SC061Q07TA',
'SC061Q08TA',
'SC061Q09TA',
'SC061Q10TA']


In [25]:
for var in vari:
    print(meta.variable_value_labels[var])

{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 

In [26]:
for var in vari:
    print(data3[var].value_counts(dropna=False))

1.0    661
2.0    384
NaN     44
Name: SC167Q01HA, dtype: int64
1.0    954
2.0     95
NaN     40
Name: SC167Q02HA, dtype: int64
1.0    796
2.0    252
NaN     41
Name: SC167Q03HA, dtype: int64
1.0    1019
NaN      39
2.0      31
Name: SC167Q04HA, dtype: int64
1.0    1040
NaN      39
2.0      10
Name: SC167Q05HA, dtype: int64
1.0    998
2.0     53
NaN     38
Name: SC167Q06HA, dtype: int64
1.0    1008
NaN      43
2.0      38
Name: SC158Q01HA, dtype: int64
1.0    923
2.0    120
NaN     46
Name: SC158Q02HA, dtype: int64
1.0    973
2.0     70
NaN     46
Name: SC158Q04HA, dtype: int64
1.0    1001
NaN      46
2.0      42
Name: SC158Q07HA, dtype: int64
1.0    918
2.0    122
NaN     49
Name: SC158Q08HA, dtype: int64
1.0    948
2.0     93
NaN     48
Name: SC158Q09HA, dtype: int64
1.0    990
2.0     56
NaN     43
Name: SC158Q12HA, dtype: int64
2.0    524
1.0    301
3.0    171
4.0     58
NaN     35
Name: SC061Q01TA, dtype: int64
2.0    546
1.0    252
3.0    195
4.0     57
NaN     39
Name: SC061Q02T

In [27]:
data4 = Transfor(vari, data3, data4)

## SC159Q01HA	Does your school host visiting teachers from other countries?
## SC003Q01TA	What is the average size of test language> classes in <national modal grade for 15-year-olds> in your school?
## SC053Q01TA	This academic year>, activities offered to national modal grade for 15-year-olds>: Band, orchestra or choir
## SC053Q02TA	This academic year>, activities offered to national modal grade for 15-year-olds>: School play or school musical
## SC053Q03TA	This academic year>, activities offered to national modal grade for 15-year-olds>: School yearbook, newspaper [...]
## SC053Q04TA	This academic year>, activities offered to national modal grade for 15-year-olds>: Volunteering [...]
## SC053Q12IA	This academic year>, activities offered to national modal grade for 15-year-olds>: Book club
## SC053Q13IA	This academic year>, activities offered to national modal grade for 15-year-olds>: Debating club or debating [...]
## SC053Q09TA	This academic year>, activities offered to national modal grade for 15-year-olds>: Art club or art activities
## SC053Q10TA	This academic year>, activities offered to national modal grade for 15-year-olds>: Sporting team or sporting [...]
## SC053Q14IA	This academic year>, activities offered to national modal grade for 15-year-olds>: Lectures and/or seminars [...]
## SC053Q15IA	This academic year>, activities offered to national modal grade for 15-year-olds>: Collaboration with local libraries
## SC053Q16IA	This academic year>, activities offered to national modal grade for 15-year-olds>: Collaboration with local newspapers
## SC053D11TA	This academic year>,follow. activities/school offersnational modal grade for 15-year-olds>? country specific item>
## SC150Q01IA	School's equity-oriented policies: These students attend regular classes and receive additional periods of [...]
## SC150Q02IA	School's equity-oriented policies: Before transferring to regular classes, [...] preparatory programme aimed at [...]
## SC150Q03IA	School's equity-oriented policies: Before transferring to regular classes, [...] instruction in school subjects [...]
## SC150Q04IA	School's equity-oriented policies: These students receive [...] amounts of instruction in their heritage language>
## SC150Q05IA	School’s equity-oriented policies: Class size is reduced to cater to the special needs of these students.
## SC164Q01HA	In the last full academic year, what proportion of students in [...] final grade left school without a certificate>?
## SC152Q01HA	Does your school offer additional test language> lessons [...] during the usual school hours?
## SC160Q01WA	What is the purpose of these additional test language> lessons?
## SC052Q01NA	For 15-year old students, school provides study help: Room(s) where the students can do their homework
## SC052Q02NA	For 15-year old students, school provides study help: Staff help with homework
## SC052Q03HA	For 15-year old students, school provides study help: Peer-to-peer tutoring
## SCHLTYPE	School Ownership
## CLSIZE	Class Size


In [28]:
vari = ['SC159Q01HA',
'SC003Q01TA',
'SC053Q01TA',
'SC053Q02TA',
'SC053Q03TA',
'SC053Q04TA',
'SC053Q12IA',
'SC053Q13IA',
'SC053Q09TA',
'SC053Q10TA',
'SC053Q14IA',
'SC053Q15IA',
'SC053Q16IA',
#'SC053D11TA',
'SC150Q01IA',
'SC150Q02IA',
'SC150Q03IA',
'SC150Q04IA',
'SC150Q05IA',
'SC052Q01NA',
'SC052Q02NA',
'SC052Q03HA',
'SC152Q01HA',
'SC160Q01WA',
#'PRIVATESCH',
'SCHLTYPE',
'CLSIZE']


In [29]:
for var in vari:
    print(meta.variable_value_labels[var])

{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: '15 students or fewer', 2.0: '16-20 students', 3.0: '21-25 students', 4.0: '26-30 students', 5.0: '31-35 students', 6.0: '36-40 students', 7.0: '41-45 students', 8.0: '46-50 students', 9.0: 'More than 50 students', 95.0: 'Valid Skip', 97.0: 'Not Applicable', 98.0: 'Invalid', 99.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0: 'No Response'}
{1.0: 'Yes', 2.0: 'No', 5.0: 'Valid Skip', 7.0: 'Not Applicable', 8.0: 'Invalid', 9.0

In [30]:
for var in vari:
    print(data3[var].value_counts(dropna=False))

2.0    604
1.0    446
NaN     39
Name: SC159Q01HA, dtype: int64
4.0    351
3.0    317
2.0    129
9.0     84
5.0     73
NaN     41
1.0     39
8.0     21
7.0     19
6.0     15
Name: SC003Q01TA, dtype: int64
2.0    724
1.0    293
NaN     72
Name: SC053Q01TA, dtype: int64
2.0    542
1.0    487
NaN     60
Name: SC053Q02TA, dtype: int64
2.0    587
1.0    433
NaN     69
Name: SC053Q03TA, dtype: int64
1.0    621
2.0    410
NaN     58
Name: SC053Q04TA, dtype: int64
2.0    605
1.0    414
NaN     70
Name: SC053Q12IA, dtype: int64
2.0    642
1.0    375
NaN     72
Name: SC053Q13IA, dtype: int64
2.0    618
1.0    392
NaN     79
Name: SC053Q09TA, dtype: int64
1.0    803
2.0    235
NaN     51
Name: SC053Q10TA, dtype: int64
1.0    874
2.0    166
NaN     49
Name: SC053Q14IA, dtype: int64
2.0    711
1.0    309
NaN     69
Name: SC053Q15IA, dtype: int64
2.0    844
1.0    175
NaN     70
Name: SC053Q16IA, dtype: int64
2.0    551
1.0    452
NaN     86
Name: SC150Q01IA, dtype: int64
2.0    728
1.0    269
NaN  

In [31]:
data4 = Transfor(vari, data3, data4)

# Añadir valores numericos, imputamos la media a los NaN

## SC016Q01TA	Percentage of total funding for school year from: Government
## SC016Q02TA	Percentage of total funding for school year from: Student fees or school charges paid by parents
## SC016Q03TA	Percentage of total funding for school year from: Benefactors, donations, bequests, sponsorships, parent fundraising
## SC016Q04TA	Percentage of total funding for school year from: Other
## SC002Q01TA	As of February 1, 2018>, what was the total school enrolment (number of students)? Number of boys
## SC002Q02TA	As of February 1, 2018>, what was the total school enrolment (number of students)? Number of girls
## SC048Q01NA	Percentage national modal grade for 15-year-olds>: Students whose heritage language> is different from test language>
## SC048Q02NA	Percentage national modal grade for 15-year-olds>: Students with special needs
## SC048Q03NA	Percentage national modal grade for 15-year-olds>: Students from socioeconomically disadvantaged homes
## SC004Q01TA	At your school, what is the total number of students in the national modal grade for 15-year-olds>?
## SC004Q02TA	Approximately, how many computers are available for these students for educational purposes?
## SC004Q03TA	Approximately, how many of these computers are connected to Internet/World Wide Web?
## SC004Q04NA	Approximately, how many of these computers are portable (e.g. laptop, tablet)?
## SC004Q05NA	Approximately, how many interactive Whiteboards are available in the school altogether?
## SC004Q06NA	Approximately, how many data projectors are available in the school altogether?
## SC004Q07NA	Approximately, how many computers with Internet connection are available for teachers in your school?
## SC018Q01TA01	Teachers in TOTAL: Full-time
## SC018Q01TA02	Teachers in TOTAL: Part-time
## SC018Q02TA01	Teachers fully certified> by the appropriate authority>: Full-time
## SC018Q02TA02	Teachers fully certified> by the appropriate authority>: Part-time
## SC018Q05NA01	Teachers with an ISCED Level 5A Bachelor degree> qualification: Full-time
## SC018Q05NA02	Teachers with an ISCED Level 5A Bachelor degree> qualification: Part-time
## SC018Q06NA01	Teachers with an ISCED Level 5A Master's degree> qualification: Full-time
## SC018Q06NA02	Teachers with an ISCED Level 5A Master's degree> qualification: Part-time
## SC018Q07NA01	Teachers with an ISCED Level 6> qualification: Full-time
## SC018Q07NA02	Teachers with an ISCED Level 6> qualification: Part-time
## SC025Q01NA	During the last three months, what percentage of teaching staff [...] attended a programme of professional development?
## SC164Q01HA	In the last full academic year, what proportion of students in [...] final grade left school without a certificate>?
## SC064Q01TA	Proportion of parents: Discussed their child's progress with a teacher on their own initiative
## SC064Q02TA	Proportion of parents: Discussed their child's progress on the initiative of one of their child's teachers
## SC064Q03TA	Proportion of parents: Participated in local school government (e.g. parent council or school management committee)
## SC064Q04NA	Proportion of parents: Volunteered in physical or extra-curricular activities [...]
## STRATIO	Student-Teacher ratio
## SCHSIZE	School Size (Sum)
## RATCMP1	Number of available computers per student at modal grade
## RATCMP2	Proportion of available computers that are connected to the Internet
## TOTAT	Total number of all teachers at school
## PROATCE	Index proportion of all teachers fully certified
## PROAT5AB	Index proportion of all teachers ISCED LEVEL 5A Bachelor
## PROAT5AM	Index proportion of all teachers ISCED LEVEL 5A Master
## PROAT6	Index proportion of all teachers ISCED LEVEL 6
## CREACTIV	Creative extra-curricular activities (Sum)


In [32]:
#Recorremos todas las variables e imputamos el valor máximo a los NaN
#def Transfor (var, data3, data4):
#    for var in vari:
#        a = data3[var].value_counts(dropna=False)
#        e = a.idxmax()
#        #Imputamos los valores nulos al máximo
#        data3.fillna({var:e}, inplace=True)
#        ##Dicotomizamos y transformamos el nombre de las columnas con el prefijo del nombre de la columna y su 'valor'
#        dummies = pd.get_dummies(data3[var])
#        for nom_col in dummies:
#            dummies = dummies.rename(columns={nom_col:var + '_' +str(nom_col)})
#        # Al tener valores si o no cruzamos por indice y unimos los dataframe 
#        data4 = pd.merge(data4, dummies, left_index=True, right_index=True)
#    return data4

In [33]:
vari = ['SC016Q01TA',
'SC016Q02TA',
'SC016Q03TA',
'SC016Q04TA',
'SC002Q01TA',
'SC002Q02TA',
'SC048Q01NA',
'SC048Q02NA',
'SC048Q03NA',
'SC004Q01TA',
'SC004Q02TA',
'SC004Q03TA',
'SC004Q04NA',
'SC004Q05NA',
'SC004Q06NA',
'SC004Q07NA',
'SC018Q01TA01',
'SC018Q01TA02',
'SC018Q02TA01',
'SC018Q02TA02',
'SC018Q05NA01',
'SC018Q05NA02',
'SC018Q06NA01',
'SC018Q06NA02',
'SC018Q07NA01',
'SC018Q07NA02',
'SC025Q01NA',
'SC164Q01HA',
'SC064Q01TA',
'SC064Q02TA',
'SC064Q03TA',
'SC064Q04NA',
'STRATIO',
'SCHSIZE',
'RATCMP1',
'RATCMP2',
'TOTAT',
'PROATCE',
'PROAT5AB',
'PROAT5AM',
'PROAT6',
'CREACTIV']


In [34]:
for var in vari:
    print(meta.variable_value_labels[var])

{995.0: 'Valid Skip', 997.0: 'Not Applicable', 998.0: 'Invalid', 999.0: 'No Response'}
{995.0: 'Valid Skip', 997.0: 'Not Applicable', 998.0: 'Invalid', 999.0: 'No Response'}
{995.0: 'Valid Skip', 997.0: 'Not Applicable', 998.0: 'Invalid', 999.0: 'No Response'}
{995.0: 'Valid Skip', 997.0: 'Not Applicable', 998.0: 'Invalid', 999.0: 'No Response'}
{99995.0: 'Valid Skip', 99997.0: 'Not Applicable', 99998.0: 'Invalid', 99999.0: 'No Response'}
{99995.0: 'Valid Skip', 99997.0: 'Not Applicable', 99998.0: 'Invalid', 99999.0: 'No Response'}
{995.0: 'Valid Skip', 997.0: 'Not Applicable', 998.0: 'Invalid', 999.0: 'No Response'}
{995.0: 'Valid Skip', 997.0: 'Not Applicable', 998.0: 'Invalid', 999.0: 'No Response'}
{995.0: 'Valid Skip', 997.0: 'Not Applicable', 998.0: 'Invalid', 999.0: 'No Response'}
{99995.0: 'Valid Skip', 99997.0: 'Not Applicable', 99998.0: 'Invalid', 99999.0: 'No Response'}
{99995.0: 'Valid Skip', 99997.0: 'Not Applicable', 99998.0: 'Invalid', 99999.0: 'No Response'}
{99995.0: '

In [35]:
for var in vari:
    print(data3[var].value_counts(dropna=False))

100.0    453
90.0      74
NaN       69
80.0      57
99.0      52
        ... 
37.0       1
39.0       1
47.0       1
79.0       1
54.0       1
Name: SC016Q01TA, Length: 62, dtype: int64
NaN      366
0.0      327
20.0      45
10.0      41
5.0       37
100.0     32
1.0       28
30.0      23
15.0      20
25.0      18
40.0      15
50.0      13
2.0       10
8.0        9
12.0       7
4.0        7
60.0       6
3.0        6
9.0        5
18.0       5
27.0       4
6.0        4
35.0       4
24.0       4
17.0       4
19.0       3
23.0       3
34.0       3
45.0       3
11.0       3
70.0       3
80.0       3
7.0        2
55.0       2
13.0       2
63.0       2
42.0       2
14.0       1
67.0       1
36.0       1
95.0       1
38.0       1
26.0       1
33.0       1
43.0       1
53.0       1
99.0       1
21.0       1
29.0       1
90.0       1
48.0       1
51.0       1
98.0       1
44.0       1
56.0       1
Name: SC016Q02TA, dtype: int64
0.0     439
NaN     417
1.0      44
10.0     38
5.0      27
20.0    

In [36]:
# detecta como outliers a aquellos valores cuya distancia a la media es mayor que 3 veces la desviación estándar.
# Imputa los valores NaN con la media

for var in vari:
    is_outlier = np.abs(data3[var] - data3[var].mean()) > 3.0*data3[var].std()
    print(pd.DataFrame(is_outlier).value_counts())
    
    #imputamos los outliers con la media más / menos tres veces la desviación estándar.
    var_mean = data3[var].mean()
    var_std  = data3[var].std()
    data3[var][is_outlier] = data3[var][is_outlier].clip(var_mean-3*var_std,
                                                             var_mean+3*var_std)

for var in vari:
    # Imputa los valores NaN con la media
    datmed = data3[var].fillna(data3[var].mean())

    data4 = pd.merge(data4, datmed, left_index=True, right_index=True)

SC016Q01TA
False         1055
True            34
dtype: int64
SC016Q02TA
False         1053
True            36
dtype: int64
SC016Q03TA
False         1073
True            16
dtype: int64
SC016Q04TA
False         1077
True            12
dtype: int64
SC002Q01TA
False         1073
True            16
dtype: int64
SC002Q02TA
False         1077
True            12
dtype: int64
SC048Q01NA
False         1089
dtype: int64
SC048Q02NA
False         1076
True            13
dtype: int64
SC048Q03NA
False         1066
True            23
dtype: int64
SC004Q01TA
False         1083
True             6
dtype: int64
SC004Q02TA
False         1069
True            20
dtype: int64
SC004Q03TA
False         1070
True            19
dtype: int64
SC004Q04NA
False         1069
True            20
dtype: int64
SC004Q05NA
False         1071
True            18
dtype: int64
SC004Q06NA
False         1076
True            13
dtype: int64
SC004Q07NA
False         1074
True            15
dtype: int64
SC018Q01TA01
False         

<ipython-input-36-3965049e4fe5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3[var][is_outlier] = data3[var][is_outlier].clip(var_mean-3*var_std,
<ipython-input-36-3965049e4fe5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3[var][is_outlier] = data3[var][is_outlier].clip(var_mean-3*var_std,
<ipython-input-36-3965049e4fe5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3[var][is_outlier] = data3[var][is_outlier]

SC018Q02TA01
False           1082
True               7
dtype: int64
SC018Q02TA02
False           1076
True              13
dtype: int64
SC018Q05NA01
False           1083
True               6
dtype: int64
SC018Q05NA02
False           1078
True              11
dtype: int64
SC018Q06NA01
False           1085
True               4
dtype: int64
SC018Q06NA02
False           1078
True              11
dtype: int64
SC018Q07NA01
False           1079
True              10
dtype: int64
SC018Q07NA02
False           1068
True              21
dtype: int64
SC025Q01NA
False         1089
dtype: int64
SC164Q01HA
False         1074
True            15
dtype: int64
SC064Q01TA
False         1089
dtype: int64
SC064Q02TA
False         1089
dtype: int64
SC064Q03TA
False         1058
True            31
dtype: int64
SC064Q04NA
False         1070
True            19
dtype: int64
STRATIO
False      1082
True          7
dtype: int64
SCHSIZE
False      1074
True         15
dtype: int64
RATCMP1
False      1085
True       

<ipython-input-36-3965049e4fe5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3[var][is_outlier] = data3[var][is_outlier].clip(var_mean-3*var_std,
<ipython-input-36-3965049e4fe5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3[var][is_outlier] = data3[var][is_outlier].clip(var_mean-3*var_std,
<ipython-input-36-3965049e4fe5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3[var][is_outlier] = data3[var][is_outlier]

In [37]:
data4.to_csv('EscuelasTran.csv')

In [38]:
## DICCIONARO VALORES
valmal = ['Valid Skip', 'Not Applicable', 'Invalid', 'No Response']
def get_key(val, my_dict):
    for key, value in my_dict.items():
         if val == value:
             return key

for variable in meta.column_names:
    if variable in meta.variable_value_labels:
        for valmal2 in valmal:
            k1 = get_key(valmal2, meta.variable_value_labels[variable])
            if k1 != None:
                del meta.variable_value_labels[variable][k1]
                
import pickle
a_file = open("valuesEsc.pkl", "wb")
pickle.dump(meta.variable_value_labels, a_file)
a_file.close()                

In [39]:
a_file = open("labelEsc.pkl", "wb")
pickle.dump(meta.column_names_to_labels, a_file)
a_file.close()

In [40]:
meta.variable_value_labels

{'CNTRYID': {8.0: 'Albania',
  31.0: 'Baku (Azerbaijan)',
  32.0: 'Argentina',
  36.0: 'Australia',
  40.0: 'Austria',
  56.0: 'Belgium',
  70.0: 'Bosnia and Herzegovina',
  76.0: 'Brazil',
  96.0: 'Brunei Darussalam',
  100.0: 'Bulgaria',
  112.0: 'Belarus',
  124.0: 'Canada',
  152.0: 'Chile',
  158.0: 'Chinese Taipei',
  170.0: 'Colombia',
  188.0: 'Costa Rica',
  191.0: 'Croatia',
  196.0: 'Cyprus',
  203.0: 'Czech Republic',
  208.0: 'Denmark',
  214.0: 'Dominican Republic',
  233.0: 'Estonia',
  246.0: 'Finland',
  250.0: 'France',
  268.0: 'Georgia',
  276.0: 'Germany',
  300.0: 'Greece',
  344.0: 'Hong Kong',
  348.0: 'Hungary',
  352.0: 'Iceland',
  360.0: 'Indonesia',
  372.0: 'Ireland',
  376.0: 'Israel',
  380.0: 'Italy',
  383.0: 'Kosovo',
  392.0: 'Japan',
  398.0: 'Kazakhstan',
  400.0: 'Jordan',
  410.0: 'Korea',
  422.0: 'Lebanon',
  428.0: 'Latvia',
  440.0: 'Lithuania',
  442.0: 'Luxembourg',
  446.0: 'Macao',
  458.0: 'Malaysia',
  470.0: 'Malta',
  484.0: 'Mexico',